Can we predict whether or not a class completed a test preparation course based on their test results?


Are high marks in school a result of hardwork and effort, or simply a result of natural intelligence? There always seem to be students who achieve high grades without having studied all semester. We want to explore whether success in exams can be achieved by natural intelligence alone, or whether hard work and studying is necessary to succeed.

We would like to try to predict the following question: Can we use the academic test scores available to us to predict whether a student has completed a test preparation course prior to their test?


We will be using a dataset of student exam scores at public schools. The dataset provides data for whether or not the student completed a test preparation course prior to the test, as well as their math, reading, and writing scores from the test. We will use this data to explore the relation between exam scores and preparation taken. 


In [9]:
library(tidyverse)

In [10]:
exam_data <- read_csv("exams.csv")
exam_data

Rows: 1000 Columns: 8
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (5): gender, race/ethnicity, parental level of education, lunch, test pr...
dbl (3): math score, reading score, writing score

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
male,group A,high school,standard,completed,67,67,63
female,group D,some high school,free/reduced,none,40,59,55
male,group E,some college,free/reduced,none,59,60,50
male,group B,high school,standard,none,77,78,68
male,group E,associate's degree,standard,completed,78,73,68
female,group D,high school,standard,none,63,77,76
female,group A,bachelor's degree,standard,none,62,59,63
male,group E,some college,standard,completed,93,88,84
male,group D,high school,standard,none,63,56,65


In [11]:
colnames(exam_data) <- make.names(colnames(exam_data))
exam_data

gender,race.ethnicity,parental.level.of.education,lunch,test.preparation.course,math.score,reading.score,writing.score
<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
male,group A,high school,standard,completed,67,67,63
female,group D,some high school,free/reduced,none,40,59,55
male,group E,some college,free/reduced,none,59,60,50
male,group B,high school,standard,none,77,78,68
male,group E,associate's degree,standard,completed,78,73,68
female,group D,high school,standard,none,63,77,76
female,group A,bachelor's degree,standard,none,62,59,63
male,group E,some college,standard,completed,93,88,84
male,group D,high school,standard,none,63,56,65


In [12]:
exam_data |>
mutate(test.preparation.course = as_factor(test.preparation.course)) |>
select(test.preparation.course, math.score, reading.score, writing.score)

test.preparation.course,math.score,reading.score,writing.score
<fct>,<dbl>,<dbl>,<dbl>
completed,67,67,63
none,40,59,55
none,59,60,50
none,77,78,68
completed,78,73,68
none,63,77,76
none,62,59,63
completed,93,88,84
none,63,56,65


In [15]:
exam_data |>
mutate(test.preparation.course = ifelse(test.preparation.course == "completed", 1, 0)) |>
select(test.preparation.course, math.score, reading.score, writing.score)

test.preparation.course,math.score,reading.score,writing.score
<dbl>,<dbl>,<dbl>,<dbl>
1,67,67,63
0,40,59,55
0,59,60,50
0,77,78,68
1,78,73,68
0,63,77,76
0,62,59,63
1,93,88,84
0,63,56,65


We expect to find that the ability does exist to predict whether or not a student has prepared for the test preparation course by their test results. The score that a student receives is a direct indication of whether or not they have prepared well by completing the test preparation course or not. We expect to find that if a student has completed the course, they have a high test score, but if they have not then their test score is more likely to be low.


These findings could show the importance of proper preparation for exams, and the impact that test preparation can have on test scores.


This could lead to future questions regarding the need for preparation prior to exams. An example could be: Can we predict how many hours a student has studied by their test results? Another stream of questioning could be: How many hours of preparation does it take to make a significant increase in an individual’s test scores?
